# Importing ARCOS Data with Dask

Last week, we used dask to play with a few datasets to get a feel for how dask works. In order to help us develop code that would run quickly, however, we worked with very small, safe datasets. 

Today, we will continue to work with dask, but this time using much larger datasets. This means that (a) doing things incorrectly may lead to your computer crashing (So save all your open files before you start!), and (b) many of the commands you are being asked run will take several minutes each. 

For familiarity, and so you can see what advantages dask can bring to your workflow, today we'll be working with the DEA ARCOS drug shipment database published by the Washington Post! However, to strike a balance between size and speed, we'll be working with a slightly thinned version that has only the last two years of data, instead of all six.

## Exercise 1

Download the thinned ARCOS data [from this link](https://www.dropbox.com/s/o7nc6yvrwog4ozi/arcos_2011_2012.tsv.zip?dl=0). It should be about 2GB zipped, 25 GB unzipped. 

## Exercise 2

Our goal today is going to be to find the pharmaceutical company that has shipped the most opioids (`MME_Conversion_Factor * CALC_BASE_WT_IN_GM`) in the US.

When working with large datasets, it is good practice to begin by prototyping your code with a subset of your data. So begin by using `pandas` to read in the first 100,000 lines of the ARCOS data and write pandas code to compute the shipments from each shipper (the group that reported the shipment). 

In [5]:
import pandas as pd
import numpy as np
import os

pd.set_option("mode.copy_on_write", True)
os.chdir("/users/nce8/downloads/")

df = pd.read_csv("arcos_2011_2012.tsv", nrows=100_000, sep="\t")
df.sample().T

/var/folders/fs/h_8_rwsn5hvg9mhp0txgc_s9v6191b/T/ipykernel_73426/2703016818.py:8: DtypeWarning: Columns (4,6,27) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("arcos_2011_2012.tsv", nrows=100_000, sep="\t")


,51243
Unnamed: 0,68243
REPORTER_DEA_NO,PL0032627
REPORTER_BUS_ACT,DISTRIBUTOR
REPORTER_NAME,AMERISOURCEBERGEN DRUG CORP
REPORTER_ADDL_CO_INFO,NaN
REPORTER_ADDRESS1,322 N 3RD STREET
REPORTER_ADDRESS2,NaN
REPORTER_CITY,PADUCAH
REPORTER_STATE,KY
REPORTER_ZIP,42001


In [6]:
df["mme"] = df["MME_Conversion_Factor"] * df["CALC_BASE_WT_IN_GM"]
grouped = df.groupby(["REPORTER_NAME"], as_index=False)["mme"].sum()
grouped.sort_values("mme", ascending=False)

,REPORTER_NAME,mme
20,MCKESSON CORPORATION,299266.331225
7,"CARDINAL HEALTH 110, LLC",54352.323711
2,AMERISOURCEBERGEN DRUG CORP,34561.394892
17,KINRAY INC,28620.315246
19,LOUISIANA WHOLESALE DRUG CO,14787.765559
11,FRANK W KERR INC,8730.016283
12,H D SMITH WHOLESALE DRUG CO,6399.324050
16,KAISER FOUNDATION HOSPITALS,3891.329580
5,BURLINGTON DRUG COMPANY,3889.490325
1,AMERICAN SALES COMPANY,3432.058005


## Exercise 3

Now let's turn to dask. Re-write your code for dask, and calculate the total shipments by reporting company. Remember: 

- Activate a conda environment with a clean dask installation.
- Start by spinning up a distributed cluster.
- Dask won't read compressed files, so you have to unzip your ARCOS data. 
- Start your cluster in a cell all by itself since you don't want to keep re-running the "start a cluster" code. 

If you need to review dask basic code, [check here](https://nickeubank.github.io/practicaldatascience_book/notebooks/PDS_not_yet_in_coursera/30_big_data/70_dask.html).

As you run your code, make sure to click on the Dashboard link below where you created your cluster:

![dask_dashboard](images/dask_cluster.png)

Among other things, the bar across the bottom should give you a sense of how long your task will take:

![dask_progress](images/dask_progress.png)

(For context, my computer (which has 10 cores) only took a couple seconds. My computer is fast, but most computers should be done within a couple minutes, tops).


In [7]:
from dask.distributed import Client

client = Client()
client

/Users/nce8/miniforge3/envs/dask/lib/python3.13/site-packages/distributed/node.py:187: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 62107 instead
  warnings.warn(


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:62107/status,
Dashboard: http://127.0.0.1:62107/status,Workers: 5
Total threads: 10,Total memory: 64.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:62108,Workers: 5
Dashboard: http://127.0.0.1:62107/status,Total threads: 10
Started: Just now,Total memory: 64.00 GiB
Comm: tcp://127.0.0.1:62126,Total threads: 2
Dashboard: http://127.0.0.1:62127/status,Memory: 12.80 GiB
Nanny: tcp://127.0.0.1:62111,


In [8]:
import dask.dataframe as dd

df = dd.read_csv(
    "/users/nce8/downloads/arcos_2011_2012.tsv",
    sep="\t",
    usecols=["REPORTER_NAME", "MME_Conversion_Factor", "CALC_BASE_WT_IN_GM"],
)

df["mme"] = df["MME_Conversion_Factor"] * df["CALC_BASE_WT_IN_GM"]
grouped = df.groupby(["REPORTER_NAME"])["mme"].sum()
results = grouped.compute()

2024-11-19 13:05:46,592 - distributed.shuffle._scheduler_plugin - WARNING - Shuffle 598788feafe30f29ea9811cb7fb2caa2 initialized by task ('shuffle-transfer-598788feafe30f29ea9811cb7fb2caa2', 378) executed on worker tcp://127.0.0.1:62132
2024-11-19 13:06:02,885 - distributed.shuffle._scheduler_plugin - WARNING - Shuffle 598788feafe30f29ea9811cb7fb2caa2 deactivated due to stimulus 'task-finished-1732039562.883873'
2024-11-19 13:06:03,561 - distributed.shuffle._scheduler_plugin - WARNING - Shuffle 942382d9008028879f257963906f7205 initialized by task ('shuffle-transfer-942382d9008028879f257963906f7205', 99) executed on worker tcp://127.0.0.1:62129
2024-11-19 13:06:21,346 - distributed.shuffle._scheduler_plugin - WARNING - Shuffle 942382d9008028879f257963906f7205 deactivated due to stimulus 'task-finished-1732039581.3442008'
2024-11-19 13:06:22,008 - distributed.shuffle._scheduler_plugin - WARNING - Shuffle 36907e7bcfab7b9ba83c9c7885ff5208 initialized by task ('shuffle-transfer-36907e7bcfab

In [9]:
results.sort_values(ascending=False) / 1_000

REPORTER_NAME
MCKESSON CORPORATION                             56046.787034
CARDINAL HEALTH                                  46719.583860
WALGREEN CO                                      41850.332089
AMERISOURCEBERGEN DRUG CORP                      25533.636319
CARDINAL HEALTH 110, LLC                          5896.801450
                                                     ...     
QUIQ, INC                                            0.000533
SOUTHERN MEDICAL LASERS DBA SML MEDICAL SALES        0.000454
GAVIS PHARMACEUTICALS, LLC                           0.000303
REMEDYREPACK                                         0.000182
MIKART                                               0.000170
Name: mme, Length: 316, dtype: float64

## Exercise 4

Now let's calculate, *for each state*, what company shipped the most pills?

Note you will quickly find that you can't sort in dask -- sorting in parallel is *really* tricky! So you'll have to work around that. Do what you need to do on the big dataset first, then compute it all so you get it as a regular pandas dataframe, then finish. 

In [10]:
import dask.dataframe as dd

df = dd.read_csv(
    "arcos_2011_2012.tsv",
    sep="\t",
    usecols=[
        "REPORTER_NAME",
        "MME_Conversion_Factor",
        "CALC_BASE_WT_IN_GM",
        "BUYER_STATE",
    ],
)

df["mme"] = df["MME_Conversion_Factor"] * df["CALC_BASE_WT_IN_GM"]
grouped = df.groupby(["REPORTER_NAME", "BUYER_STATE"])["mme"].sum().reset_index()
results = grouped.compute()

In [11]:
results = results.sort_values(["mme"], ascending=False)
state_maxes = results.groupby(["BUYER_STATE"]).nth(0)
state_maxes

,REPORTER_NAME,BUYER_STATE,mme
109,WALGREEN CO,FL,6.566340e+06
3,MCKESSON CORPORATION,CA,5.152963e+06
0,"CARDINAL HEALTH 110, LLC",NY,4.014733e+06
2,MCKESSON CORPORATION,PA,3.637664e+06
118,CARDINAL HEALTH,OH,2.966956e+06
11,WALGREEN CO,TX,2.318831e+06
12,WALGREEN CO,AZ,2.312494e+06
6,CARDINAL HEALTH,NC,2.273949e+06
2,MCKESSON CORPORATION,MD,2.230794e+06
102,MCKESSON CORPORATION,AL,2.143844e+06


Does this seem like a situation where a single company is responsible for the opioid epidemic?

## Exercise 5 

Now go ahead and try and re-do the importation of ARCOS data you did by hand for your opioid analysis project (In other words, for each year, calculate the total morphine equivalents sent to each county in the US, but do it with this 2 years of data). 

Rather than chunking by hand — as you should have done previously for the opioid analysis project — now let dask do all that hard work automatically.  

In [12]:
import dask.dataframe as dd

df = dd.read_csv(
    "arcos_2011_2012.tsv",
    sep="\t",
    usecols=[
        "MME_Conversion_Factor",
        "CALC_BASE_WT_IN_GM",
        "BUYER_STATE",
        "BUYER_COUNTY",
        "TRANSACTION_DATE",
    ],
    dtype={"TRANSACTION_DATE": "float64"},
)

In [13]:
df["year"] = df.TRANSACTION_DATE % 10_000
df["mme"] = df["MME_Conversion_Factor"] * df["CALC_BASE_WT_IN_GM"]
grouped = df.groupby(["BUYER_COUNTY", "BUYER_STATE", "year"])["mme"].sum().reset_index()
results = grouped.compute()

In [14]:
results.head()

,BUYER_COUNTY,BUYER_STATE,year,mme
0,BOUNDARY,ID,2011.0,4978.404375
1,BRADFORD,PA,2012.0,24691.530015
2,BRISTOL,VA,2012.0,17696.184008
3,BROOMFIELD,CO,2011.0,31757.599130
4,COLUMBIA,OR,2012.0,22251.313377


## Exercise 6

Now, re-write your opioid project's initial opioid import using dask. Each person on your team should create a NEW branch to try this. The person who wrote the initial chunking code can help everyone else understand what they did originally and the data, but everyone should write their own code. 

**WARNING:** You will probably run into a lot of type errors (depending on how the ARCOS data has changed since last year). With real world messy data one of the biggest problems with dask is that it struggles if halfway through dataset it discovers that the column it *thought* was floats contains text. That's why, in the dask reading, [I specified the column type for so many columns](https://nickeubank.github.io/practicaldatascience_book/notebooks/PDS_not_yet_in_coursera/30_big_data/70_dask.html#what-can-dask-do-for-me) as `objects` explicitly. Then, because occasionally there data cleanliness issues, I had to do some converting data types by hand. 